In [27]:
# import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from skimage.io import imshow
from skimage.io import imshow, imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

import dill as pickle

np.random.seed(33)

In [5]:
df_test = pd.read_csv('../data/test_labels.csv')
df_train = pd.read_csv('../data/train_labels.csv')

bw_loaded = np.load('../data/bw_images.npz')
X_train= bw_loaded['a']
X_test = bw_loaded['b']

X_test = X_test.reshape(-1,80,60,1)
X_train = X_train.reshape(-1,80,60,1)

# Use this notebook to train 4 separate CNN classifiers. 
Each model predicts sub-categories with a master-category. The code for each model functions independently, so I have divided the notebook into sections. Also, note that for each model, you have the option of running from the top of the section (to replicate the model-training process) or to skip add and load my saved model. Feel free to pick and choose what to run.

## Section 1: Predict Sub-Categories for Apparel
Skip ahead to view results without training model

In [8]:
app_train = df_train[df_train.masterCategory=='Apparel']
app_train_idx = list(app_train.index)
X_train_app = X_train[app_train_idx]

app_test = df_test[df_test.masterCategory=='Apparel']
app_test_idx = list(app_test.index)
X_test_app = X_test[app_test_idx]

y_train_app0 = app_train.subCategory.copy().astype('category').cat.codes
y_test_app0 = app_test.subCategory.copy().astype('category').cat.codes

In [9]:
from tensorflow.keras.utils import to_categorical
y_train_app = to_categorical(y_train_app0)
y_test_app = to_categorical(y_test_app0)

In [11]:
print(y_train_app.shape)
print(y_test_app.shape)

(10394, 6)
(2603, 6)


In [21]:
cnn_app = Sequential()

In [22]:
cnn_app.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_app.add(MaxPooling2D((2, 2)))
cnn_app.add(Dropout(0.2))

cnn_app.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_app.add(MaxPooling2D(pool_size=(2, 2)))
cnn_app.add(Dropout(0.25))

cnn_app.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_app.add(MaxPooling2D(pool_size=(2, 2)))
cnn_app.add(Dropout(0.4))

cnn_app.add(Flatten())

cnn_app.add(Dense(128, activation='relu'))
cnn_app.add(Dropout(0.3))
cnn_app.add(Dense(6, activation='softmax'))

In [23]:
cnn_app.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 11, 128)      

In [24]:
cnn_app.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_app.fit(X_train_app, y_train_app, epochs=10, validation_data=(X_test_app, y_test_app), verbose=1)

In [20]:
# save the fitted model
cnn_app.save('cnn-apparel.h5')

### To replicate Apparel results without fitting model, start here : 

In [28]:
### Skip this cell if you started from beginning of notebook
cnn_app = load_model('../Results/json/cnn-apparel.h5')

In [29]:
y_true_app = y_test_app0

In [31]:
# get the predictions for the test data
predicted_classes = cnn_app.predict_classes(X_test_app)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==y_true_app)[0]
incorrect = np.nonzero(predicted_classes!=y_true_app)[0]

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [32]:
predicted_classes

array([5, 0, 5, ..., 5, 1, 5])

In [33]:
app_train.subCategory.value_counts().index
app_list = ['Bottomwear', 'Innerwear','Loungewear', 'Socks', 'Saree','Topwear']

In [36]:
target_names = ["Class {}: {}".format(i, ix) for i, ix in enumerate(app_list)]
print(classification_report(y_true_app, predicted_classes, target_names=target_names))

                     precision    recall  f1-score   support

Class 0: Bottomwear       0.94      0.97      0.95       364
 Class 1: Innerwear       0.98      0.94      0.96       329
Class 2: Loungewear       0.88      0.59      0.70        95
     Class 3: Socks       0.98      0.99      0.98        92
     Class 4: Saree       0.98      0.92      0.95       134
   Class 5: Topwear       0.97      0.99      0.98      1589

           accuracy                           0.97      2603
          macro avg       0.95      0.90      0.92      2603
       weighted avg       0.96      0.97      0.96      2603



## Section 2: Predict Sub-Categories for Accessories

In [9]:
acc_train = df_train[df_train.masterCategory=='Accessories']
acc_train_idx = list(acc_train.index)
X_train_acc = X_train[acc_train_idx]

acc_test = df_test[df_test.masterCategory=='Accessories']
acc_test_idx = list(acc_test.index)
X_test_acc = X_test[acc_test_idx]

y_train_acc0 = acc_train.subCategory.copy().astype('category').cat.codes
y_test_acc0 = acc_test.subCategory.copy().astype('category').cat.codes

# from tensorflow.keras.utils import to_categorical
# y_train_acc = to_categorical(y_train_acc0)
# y_test_acc = to_categorical(y_test_acc0)

# print(y_train_acc.shape)
# print(y_test_acc.shape)

In [10]:
acc_train.subCategory.value_counts()

Bags         2335
Watches      1916
Eyewear       834
Jewellery     821
Headwear      218
Ties          204
Name: subCategory, dtype: int64

In [29]:
cnn_acc = models.Sequential()

In [30]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_acc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_acc.add(layers.MaxPooling2D((2, 2)))
cnn_acc.add(Dropout(0.2))

cnn_acc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_acc.add(Dropout(0.25))

cnn_acc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(Dropout(0.4))


cnn_acc.add(Flatten())

cnn_acc.add(Dense(128, activation='relu'))
cnn_acc.add(Dropout(0.3))
cnn_acc.add(Dense(6, activation='softmax'))

In [31]:
cnn_acc.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 11, 128)      

In [32]:
cnn_acc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
cnn_acc.fit(X_train_acc, y_train_acc, epochs=10, validation_data=(X_test_acc, y_test_acc), verbose=1)

Train on 6328 samples, validate on 1626 samples
Epoch 1/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 6.9515 - acc: 0.6683 - val_loss: 0.5300 - val_acc: 0.8764
Epoch 2/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.5127 - acc: 0.8360 - val_loss: 0.3220 - val_acc: 0.9090
Epoch 3/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.3491 - acc: 0.8916 - val_loss: 0.3581 - val_acc: 0.8745
Epoch 4/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.2593 - acc: 0.9223 - val_loss: 0.2947 - val_acc: 0.8973
Epoch 5/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.2080 - acc: 0.9377 - val_loss: 0.2836 - val_acc: 0.9194
Epoch 6/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.1978 - acc: 0.9420 - val_loss: 0.2143 - val_acc: 0.9403
Epoch 7/10
6328/6328 [==============================] - 9s 1ms/sample - loss: 0.1741 - acc: 0.9483 - val_loss: 0.3237 - val_acc: 0.9422


In [36]:
cnn_acc.save('cnn-accessories.h5')

### To replicate Apparel results without fitting model, start here : 

In [28]:
### Skip this cell if you started from beginning of notebook
cnn_acc = load_model('../Results/json/cnn-accessories.h5')

In [37]:
y_true_acc = y_test_acc0

In [38]:
# get the predictions for the test data
predicted_classes_acc = cnn_acc.predict_classes(X_test_acc)

# get the indices to be plotted
correct = (predicted_classes_acc ==y_true_acc).to_numpy().nonzero()[0]
incorrect = (predicted_classes_acc !=y_true_acc).to_numpy().nonzero()[0]

In [39]:
from sklearn.metrics import classification_report
target_names_acc = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true_acc, predicted_classes_acc, target_names=target_names_acc))

              precision    recall  f1-score   support

     Class 0       0.97      0.99      0.98       598
     Class 1       0.98      1.00      0.99       220
     Class 2       0.96      0.92      0.94        51
     Class 3       0.99      0.92      0.95       216
     Class 4       1.00      1.00      1.00        43
     Class 5       0.99      0.99      0.99       498

    accuracy                           0.98      1626
   macro avg       0.98      0.97      0.98      1626
weighted avg       0.98      0.98      0.98      1626



## Section 3: Predict Sub-Categories for Footwear

In [20]:
fw_train = df_train[df_train.masterCategory=='Footwear']
fw_train_idx = list(fw_train.index)
X_train_fw = X_train[fw_train_idx]

fw_test = df_test[df_test.masterCategory=='Footwear']
fw_test_idx = list(fw_test.index)
X_test_fw = X_test[fw_test_idx]

y_train_fw0 = fw_train.subCategory.copy().astype('category').cat.codes
y_test_fw0 = fw_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_fw = to_categorical(y_train_fw0)
y_test_fw = to_categorical(y_test_fw0)

print(y_train_fw.shape)
print(y_test_fw.shape)

(7090, 3)
(1826, 3)


In [21]:
cnn_fw = models.Sequential()

In [22]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_fw.add(layers.MaxPooling2D((2, 2)))
cnn_fw.add(Dropout(0.2))

# cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
# cnn_fw.add(Dropout(0.25))

cnn_fw.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
cnn_fw.add(Dropout(0.2))

cnn_fw.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(Dropout(0.4))


cnn_fw.add(Flatten())
# cnn_fw.add(Dense(512, activation='relu'))
# cnn_fw.add(Dropout(0.5))
cnn_fw.add(Dense(128, activation='relu'))
cnn_fw.add(Dropout(0.3))
cnn_fw.add(Dense(3, activation='softmax'))

In [23]:
cnn_fw.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 11, 128)      

In [24]:
cnn_fw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
cnn_fw.fit(X_train_fw, y_train_fw, epochs=10, validation_data=(X_test_fw, y_test_fw), verbose=1)

Train on 7090 samples, validate on 1826 samples
Epoch 1/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.6567 - acc: 0.8023 - val_loss: 0.5542 - val_acc: 0.7990
Epoch 2/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.5384 - acc: 0.8090 - val_loss: 0.4300 - val_acc: 0.8434
Epoch 3/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.4515 - acc: 0.8295 - val_loss: 0.3585 - val_acc: 0.8773
Epoch 4/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.4148 - acc: 0.8501 - val_loss: 0.3179 - val_acc: 0.8872
Epoch 5/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.3598 - acc: 0.8664 - val_loss: 0.3024 - val_acc: 0.8817
Epoch 6/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.3291 - acc: 0.8774 - val_loss: 0.2732 - val_acc: 0.9036
Epoch 7/10
7090/7090 [==============================] - 10s 1ms/sample - loss: 0.3198 - acc: 0.8832 - val_loss: 0.2496 - val_acc: 

In [46]:
cnn_fw.save('cnn-fw.h5')

### To replicate Apparel results without fitting model, start here : 

In [28]:
### Skip this cell if you started from beginning of notebook
cnn_fw = load_model('../Results/json/cnn-fw.h5')

In [47]:
y_true_fw = y_test_fw0

In [48]:
# get the predictions for the test data
predicted_classes_fw = cnn_fw.predict_classes(X_test_fw)

# get the indices to be plotted
correct = (predicted_classes_fw ==y_true_fw).to_numpy().nonzero()[0]
incorrect = (predicted_classes_fw !=y_true_fw).to_numpy().nonzero()[0]

In [49]:
from sklearn.metrics import classification_report
target_names_fw = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_true_fw, predicted_classes_fw, target_names=target_names_fw))

              precision    recall  f1-score   support

     Class 0       0.80      0.88      0.84       177
     Class 1       0.93      0.76      0.84       206
     Class 2       0.97      0.99      0.98      1443

    accuracy                           0.95      1826
   macro avg       0.90      0.87      0.88      1826
weighted avg       0.95      0.95      0.95      1826



## Section 4: Predict Sub-Categories for Personal Care

In [6]:
pc_train = df_train[df_train.masterCategory=='Personal Care']
pc_train_idx = list(pc_train.index)
X_train_pc = X_train[pc_train_idx]

pc_test = df_test[df_test.masterCategory=='Personal Care']
pc_test_idx = list(pc_test.index)
X_test_pc = X_test[pc_test_idx]

y_train_pc0 = pc_train.subCategory.copy().astype('category').cat.codes
y_test_pc0 = pc_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_pc = to_categorical(y_train_pc0)
y_test_pc = to_categorical(y_test_pc0)

print(y_train_pc.shape)
print(y_test_pc.shape)


(1627, 4)
(385, 4)


In [7]:
cnn_pc = models.Sequential()

In [8]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_pc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_pc.add(layers.MaxPooling2D((2, 2)))
cnn_pc.add(Dropout(0.2))

cnn_pc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_pc.add(Dropout(0.25))

cnn_pc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(Dropout(0.4))
           
cnn_pc.add(Flatten())
           
cnn_pc.add(Dense(128, activation='relu'))
cnn_pc.add(Dropout(0.3))
cnn_pc.add(Dense(4, activation='softmax'))

W0731 19:33:05.067403 139957497603904 deprecation.py:506] From /home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
cnn_pc.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 29, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 11, 128)       7

In [10]:
cnn_pc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
cnn_pc.fit(X_train_pc, y_train_pc, epochs=10, validation_data=(X_test_pc, y_test_pc), verbose=1)

Train on 1627 samples, validate on 385 samples
Epoch 1/10
1627/1627 [==============================] - 5s 3ms/sample - loss: 14.0255 - acc: 0.3669 - val_loss: 1.3457 - val_acc: 0.5013
Epoch 2/10
1627/1627 [==============================] - 3s 2ms/sample - loss: 1.2396 - acc: 0.4782 - val_loss: 1.2450 - val_acc: 0.5195
Epoch 3/10
1627/1627 [==============================] - 2s 1ms/sample - loss: 1.1478 - acc: 0.5366 - val_loss: 1.1202 - val_acc: 0.5896
Epoch 4/10
1627/1627 [==============================] - 2s 1ms/sample - loss: 1.0968 - acc: 0.5587 - val_loss: 0.8545 - val_acc: 0.6779
Epoch 5/10
1627/1627 [==============================] - 2s 1ms/sample - loss: 0.9752 - acc: 0.6146 - val_loss: 0.8080 - val_acc: 0.7013
Epoch 6/10
1627/1627 [==============================] - 2s 1ms/sample - loss: 0.9246 - acc: 0.6478 - val_loss: 0.7725 - val_acc: 0.7247
Epoch 7/10
1627/1627 [==============================] - 2s 1ms/sample - loss: 0.8257 - acc: 0.6699 - val_loss: 0.6095 - val_acc: 0.7870


In [12]:
cnn_pc.save('cnn-pc.h5')

### To replicate Apparel results without fitting model, start here : 

In [28]:
### Skip this cell if you started from beginning of notebook
cnn_pc = load_model('../Results/json/cnn-pc.h5')

In [13]:
y_true_pc = y_test_pc0

In [14]:
# get the predictions for the test data
predicted_classes_pc = cnn_pc.predict_classes(X_test_pc)

# get the indices to be plotted
correct = (predicted_classes_pc ==y_true_pc).to_numpy().nonzero()[0]
incorrect = (predicted_classes_pc !=y_true_pc).to_numpy().nonzero()[0]

In [19]:
pc_list = list(pc_test.subCategory.value_counts().index)
from sklearn.metrics import classification_report
target_names_pc = ["Class {}: {}".format(i, name) for i, name in enumerate(pc_list)]
print(classification_report(y_true_pc, predicted_classes_pc, target_names=target_names_pc))

                    precision    recall  f1-score   support

Class 0: Fragrance       0.90      0.96      0.93       191
     Class 1: Lips       0.89      1.00      0.94        93
    Class 2: Nails       1.00      0.34      0.51        35
   Class 3: Makeup       0.94      0.92      0.93        66

          accuracy                           0.91       385
         macro avg       0.93      0.81      0.83       385
      weighted avg       0.91      0.91      0.89       385

